# Data Preprocess 1

This journal describes how to prepare the data to run the RELOG scenarios. He we will cover:
1. Waste files generation. Correlates latitude and longitude with a region and state. Takes PV ICE-generated data file and creates individual waste files based on the type of waste.
2. Make initial amounts file. Using the previously generated files, we show how to make an 'Initial amounts file' so it is in RELOG input data format. 
3. Make recycling candidates files. Generate the PV recycling plant candidates files using the correct area cost factor.
4. Make a shankey diagram from section 1.
5. Render a cloropeth map from section 2 (not finished!)

A continuation of this journal can be found in [Data Preprocess 2](./data_preprocess_2.ipynb).

***NOTE:** All quantities are given in **metric tonnes**.*

## 0. Load necessary libraries

In [1]:
import numpy as np
import pandas as pd
import os,sys
import matplotlib.pyplot as plt
from pathlib import Path

from geopy.geocoders import Nominatim
from geopy.point import Point
# initialize Nominatim API
geolocator = Nominatim(user_agent="geoapiExercises")

---
## 1. Waste files generation

### 1.1. Get the GIS locations from the latitude and longitude

Filter the USA areas.

In [2]:
GISfile = str(Path().resolve().parent.parent.parent / 'gis_centroid_n.xlsx')
GIS = pd.read_excel(GISfile)
GIS_us = GIS[GIS.country == 'USA']
GIS_us.reset_index(inplace=True)
GIS_us = GIS_us.iloc[0:134]
GIS_us.to_csv('GIS_us_collection_centers_only.csv')

In [3]:
def city_state_country(row):
    # I map latitude and longitude with actual names of those places.
    coord = f"{row['lat']}, {row['long']}"
    location = geolocator.reverse(coord, exactly_one=True)
    address = location.raw['address']
    city = address.get('city', '')
    county = address.get('county', '')
    state = address.get('state', '')
    country = address.get('country', '')
    county_state = address.get('county', 'state')
    row['city'] = city
    row['county'] = county
    row['state'] = state
    row['country'] = country
    row['location'] = county_state
    return row

Generate a GIS file with the name of the locations, this will be useful for section 2, 3 and 4. Where the location names matter to assign area cost factors and to locate waste in a map. I would recommend running this function once, saving the output, and then, when needed, just read the generated csv.

In [4]:
# This one takes a long time!! Apply the previous function to obtain the names of the locations.
GIS = GIS.apply(city_state_country, axis=1)

In [7]:
GIS.to_csv('gis_region_names.csv')

In [6]:
GIS_us_long_lat = GIS_us[['long', 'lat']]
GIS_us_id = GIS_us[['id']]

### 1.2. Correlate GIS longitude and latitude with FIPS codes for the map diagram

In [18]:
fips_county_codes = []
fips_state_codes = []

In [19]:
# Code from https://gis.stackexchange.com/questions/294641/python-code-for-transforming-lat-long-into-fips-codes
import requests
import urllib

#Encode parameters 
for lon, lat in GIS_us_long_lat.itertuples(index=False):
    params = urllib.parse.urlencode({'latitude': lat, 'longitude':lon, 'format':'json'})
    #Contruct request URL
    url = 'https://geo.fcc.gov/api/census/block/find?' + params

    #Get response from API
    response = requests.get(url)

    #Parse json in response
    data = response.json()
    fips_county_codes.append(data['County']['FIPS'])
    fips_state_codes.append(data['State']['FIPS'])
    #Print FIPS code

In [20]:
data

{'Block': {'FIPS': '511455004001005',
  'bbox': [-78.080715, 37.624393, -78.006353, 37.65739]},
 'County': {'FIPS': '51145', 'name': 'Powhatan County'},
 'State': {'FIPS': '51', 'code': 'VA', 'name': 'Virginia'},
 'status': 'OK',
 'executionTime': '0'}

In [21]:
GIS_us_long_lat['fips_county'] = fips_county_codes
GIS_us_long_lat['fips_state'] = fips_state_codes

/var/folders/w2/85_h_nyn0mlbxf81x3k9n53xh90n_1/T/ipykernel_16339/3894708014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GIS_us_long_lat['fips_county'] = fips_county_codes
/var/folders/w2/85_h_nyn0mlbxf81x3k9n53xh90n_1/T/ipykernel_16339/3894708014.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GIS_us_long_lat['fips_state'] = fips_state_codes


In [22]:
GIS_us_long_lat.to_csv('GIS_us_long_lat.csv')

,long,lat,fips_county,fips_state
0,-121.452252,47.820991,53061,53
1,-117.159039,35.120104,06071,06
2,-78.257714,38.791108,51187,51
3,-82.192477,28.708695,12119,12
4,-80.563731,26.677092,12099,12
...,...,...,...,...
129,-82.058502,34.683402,45059,45
130,-80.470185,33.666562,45027,45
131,-81.469485,35.703545,37023,37
132,-78.063174,35.438992,37191,37


### 1.3. Separate the PV ICE output and create individual material file

In [23]:
cwd = os.getcwd()
pvice_folder = os.path.join(cwd, 'PV_ICE_outputs')

In [24]:
csi_eol = pd.read_csv(os.path.join(pvice_folder, 'PVICE_RELOG_PCA_cSi_WasteEOL.csv'), index_col='year')
cdte_eol = pd.read_csv(os.path.join(pvice_folder, 'PVICE_RELOG_PCA_CdTe_WasteEOL.csv'), index_col='year')

In [25]:
print('We have %s collection centers.' % len(GIS_us))

We have 134 collection centers.


Now I need to select the columns and separate them by material, then add a column identifying the locations. Ideally, I need to populate the table for the material

In [26]:
material_list_csi = ['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet', 'Module']
material_list_cdte = ['cadmium', 'tellurium', 'glass_cdte', 'aluminium_frames_cdte', 'Module', 'copper_cdte', 'encapsulant_cdte']

In [27]:
nums = np.arange(1,42)
years = np.arange(2010,2051)
years_dict = {nums[i]: years[i] for i in range(len(nums))}

In [28]:
mats = ['csi', 'cdte']

In [29]:
GIS_us_long_lat

,long,lat,fips_county,fips_state
0,-121.452252,47.820991,53061,53
1,-117.159039,35.120104,06071,06
2,-78.257714,38.791108,51187,51
3,-82.192477,28.708695,12119,12
4,-80.563731,26.677092,12099,12
...,...,...,...,...
129,-82.058502,34.683402,45059,45
130,-80.470185,33.666562,45027,45
131,-81.469485,35.703545,37023,37
132,-78.063174,35.438992,37191,37


In [30]:
for y in mats:
    if y == 'csi':
        for x in material_list_csi:
            globals()['%s_%s_sel' % (y, x)] = [col for col in globals()['%s_eol' % y].columns if x in col]
            globals()['%s_%s' % (y, x)] = csi_eol.filter(globals()['%s_%s_sel' % (y, x)], axis=1)
            globals()['%s_%s' % (y, x)] = globals()['%s_%s' % (y, x)].transpose()
            globals()['%s_%s' % (y, x)].reset_index(inplace=True)
            globals()['%s_%s' % (y, x)] = pd.concat([globals()['%s_%s' % (y, x)], GIS_us_long_lat], axis=1, ignore_index=True)
            globals()['%s_%s' % (y, x)].rename(columns = years_dict, inplace=True)
           # globals()['%s_%s' % (y, x)].rename(columns = {42:'PCA area'}, inplace=True)
            globals()['%s_%s' % (y, x)].rename(columns = {42:'longitude', 43:'latitude', 44:'FIPS'}, inplace=True)
            globals()['%s_%s' % (y, x)]['total waste'] = globals()['%s_%s' % (y, x)].loc[:, 2010:2050].sum(axis=1)
            globals()['%s_%s' % (y, x)].to_csv('{}_wasteEOL_{}.csv'.format(y, x), index=False)
    elif y == 'cdte':
        for x in material_list_cdte:
            globals()['%s_%s_sel' % (y, x)] = [col for col in globals()['%s_eol' % y].columns if x in col]
            globals()['%s_%s' % (y, x)] = cdte_eol.filter(globals()['%s_%s_sel' % (y, x)], axis=1)
            globals()['%s_%s' % (y, x)] = globals()['%s_%s' % (y, x)].transpose()
            globals()['%s_%s' % (y, x)].reset_index(inplace=True)
            globals()['%s_%s' % (y, x)] = pd.concat([globals()['%s_%s' % (y, x)], GIS_us_long_lat], axis=1, ignore_index=True)
            globals()['%s_%s' % (y, x)].rename(columns = years_dict, inplace=True)
           # globals()['%s_%s' % (y, x)].rename(columns = {42:'PCA area'}, inplace=True)
            globals()['%s_%s' % (y, x)].rename(columns = {42:'longitude', 43:'latitude', 44:'FIPS'}, inplace=True)
            globals()['%s_%s' % (y, x)]['total waste'] = globals()['%s_%s' % (y, x)].loc[:, 2010:2050].sum(axis=1)
            globals()['%s_%s' % (y, x)].to_csv('{}_wasteEOL_{}.csv'.format(y, x), index=False)

## 2. Make the initial amounts file

Here I will be generating the file that has the waste initial amounts according to RELOG's template (Initial amounts - Template.csv).
![image.png](Initial_amounts_temp.png)

I have two options:
1) Add at the wastes and then do the processing to create ONE template of initial amounts.
2) Create THREE templates, two with CdTe and cSi separated, and then create another one summing the previous two.

I am going to do option 2, that way if we decide to generate a separate RELOG scenario with one or the other technology, we have already separate files. I am also  going to create the files for individual materials in case they update the software so we also have material timeseries.

### 2.1. Load the waste files that we just generated.

**LOAD OPTION 1:** If you have run Section 1 cells, fetch the data as csi_Module and cdte_Module. I make a copy of these files since I will be changing them considerably.

In [45]:
csi_Module_ia = csi_Module.copy()
csi_glass_ia = csi_glass.copy()
csi_silicon_ia = csi_silicon.copy()
csi_silver_ia = csi_silver.copy()
csi_copper_ia = csi_copper.copy()
csi_aluminium_frames_ia = csi_aluminium_frames.copy()
csi_encapsulant_ia = csi_encapsulant.copy()
csi_backsheet_ia = csi_backsheet.copy()


cdte_Module_ia = cdte_Module.copy()
cdte_cadmium_ia = cdte_cadmium.copy()
cdte_tellurium_ia = cdte_tellurium.copy()
cdte_glass_cdte_ia = cdte_glass_cdte.copy()
cdte_aluminium_frames_cdte_ia = cdte_aluminium_frames_cdte.copy()
cdte_copper_cdte_ia = cdte_copper_cdte.copy()
cdte_encapsulant_cdte_ia = cdte_encapsulant_cdte.copy()

# 'ia' stands for initial amounts

**LOAD OPTION 2:** If you haven't run Section 1 cells, use function load_csv with the corresponding file name.

In [46]:
# Uncomment if you need this option
# csi_Module = pd.read_csv('csi_wasteEOL_Module.csv')
# cdte_Module = pd.read_csv('cdte_wasteEOL_Module.csv')

### 2.2. Drop unnecessary columns

Drop the 0, 2010 (there is no waste here), FIPS, 45, longitude, latitude and total waste columns. Then I insert the 'name' row and then move longitude and latitude rows

In [47]:
csi_Module_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
csi_glass_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
csi_silicon_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
csi_silver_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
csi_copper_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
csi_aluminium_frames_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
csi_encapsulant_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
csi_backsheet_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.

In [48]:
cdte_Module_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
cdte_cadmium_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
cdte_tellurium_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
cdte_glass_cdte_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.)
cdte_aluminium_frames_cdte_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
cdte_copper_cdte_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.
cdte_encapsulant_cdte_ia.drop([0, 2010, 'FIPS', 45, 'total waste', 'latitude', 'longitude'], axis=1, inplace= True) # Run this one only once or it will throw an error.

### 2.3. Add the name locations, latitude and longitude with the right names to the right position.

I take the location file from GIS.

In [49]:
GIS_usa = GIS[GIS.country == 'United States']
GIS_usa.reset_index(inplace=True)
GIS_usa = GIS_usa.iloc[0:134] # I slice it until 142 because the next locations are not in ReEDS.

In [50]:
csi_Module_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_Module_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_Module_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

csi_glass_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_glass_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_glass_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

csi_silicon_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_silicon_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_silicon_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

csi_silver_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_silver_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_silver_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

csi_copper_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_copper_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_copper_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

csi_aluminium_frames_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_aluminium_frames_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_aluminium_frames_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

csi_encapsulant_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_encapsulant_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_encapsulant_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

csi_backsheet_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
csi_backsheet_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
csi_backsheet_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

In [51]:
cdte_Module_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
cdte_Module_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
cdte_Module_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

cdte_cadmium_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
cdte_cadmium_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
cdte_cadmium_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

cdte_tellurium_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
cdte_tellurium_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
cdte_tellurium_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

cdte_glass_cdte_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
cdte_glass_cdte_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
cdte_glass_cdte_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

cdte_aluminium_frames_cdte_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
cdte_aluminium_frames_cdte_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
cdte_aluminium_frames_cdte_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

cdte_copper_cdte_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
cdte_copper_cdte_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
cdte_copper_cdte_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

cdte_encapsulant_cdte_ia.insert(0, 'name', GIS_usa[['location']]) # Run this one only once or it will throw an error.
cdte_encapsulant_cdte_ia.insert(1, 'latitude (deg)', GIS_usa[['lat']]) 
cdte_encapsulant_cdte_ia.insert(2, 'longitude (deg)', GIS_usa[['long']]) 

#### 2.3.1 (Optional/to be decided) Add accumulated waste until year 2023

In [52]:
csi_Module_ia_sumyears = csi_Module_ia.loc[:, 2011:2023].sum(axis=1)
csi_glass_ia_sumyears = csi_glass_ia.loc[:, 2011:2023].sum(axis=1)
csi_silicon_ia_sumyears = csi_silicon_ia.loc[:, 2011:2023].sum(axis=1)
csi_silver_ia_sumyears = csi_silver_ia.loc[:, 2011:2023].sum(axis=1)
csi_copper_ia_sumyears = csi_copper_ia.loc[:, 2011:2023].sum(axis=1)
csi_aluminium_frames_ia_sumyears = csi_aluminium_frames_ia.loc[:, 2011:2023].sum(axis=1)
csi_encapsulant_ia_sumyears = csi_encapsulant_ia.loc[:, 2011:2023].sum(axis=1)
csi_backsheet_ia_sumyears = csi_backsheet_ia.loc[:, 2011:2023].sum(axis=1)


cdte_Module_ia_sumyears = cdte_Module_ia.loc[:, 2011:2023].sum(axis=1)
cdte_cadmium_ia_sumyears = cdte_cadmium_ia.loc[:, 2011:2023].sum(axis=1)
cdte_tellurium_ia_sumyears = cdte_tellurium_ia.loc[:, 2011:2023].sum(axis=1)
cdte_glass_cdte_ia_sumyears = cdte_glass_cdte_ia.loc[:, 2011:2023].sum(axis=1)
cdte_aluminium_frames_cdte_ia_sumyears = cdte_aluminium_frames_cdte_ia.loc[:, 2011:2023].sum(axis=1)
cdte_copper_cdte_ia_sumyears = cdte_copper_cdte_ia.loc[:, 2011:2023].sum(axis=1)
cdte_encapsulant_cdte_ia_sumyears = cdte_encapsulant_cdte_ia.loc[:, 2011:2023].sum(axis=1)

In [53]:
csi_Module_ia
csi_glass_ia
csi_silicon_ia
csi_silver_ia
csi_copper_ia
csi_aluminium_frames_ia
csi_encapsulant_ia
csi_backsheet_ia

cdte_Module_ia
cdte_cadmium_ia
cdte_tellurium_ia
cdte_glass_cdte_ia
cdte_aluminium_frames_cdte_ia
cdte_copper_cdte_ia
cdte_encapsulant_cdte_ia

,name,latitude (deg),longitude (deg),2011,2012,2013,2014,2015,2016,2017,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,Snohomish County,47.820991,-121.452252,3.193918e-12,7.089638e-10,4.304089e-08,5.858025e-07,3.788957e-06,1.600914e-05,5.166969e-05,...,4.225937,3.603057,1.433767,1.957628,2.638610,3.513321,4.625574,6.028302,7.785236,9.970716
1,state,35.120104,-117.159039,1.004440e-07,9.567703e-06,1.321375e-04,8.233185e-04,3.342137e-03,1.040533e-02,2.712618e-02,...,569.813433,232.343159,50.796752,64.018787,79.043556,95.607059,113.350480,131.926967,151.159163,171.195973
2,Warren County,38.791108,-78.257714,0.000000e+00,0.000000e+00,0.000000e+00,1.084450e-12,1.741209e-10,3.701481e-09,3.419047e-08,...,0.243191,1.305137,0.153481,0.189806,0.230010,0.273501,0.320117,0.370942,0.429426,0.502632
3,Sumter County,28.708695,-82.192477,4.919949e-09,4.692309e-07,6.528567e-06,4.113722e-05,1.689599e-04,5.307845e-04,1.387715e-03,...,54.809674,84.644900,28.336043,35.553030,43.629739,52.334157,61.348384,70.326498,78.990640,87.242911
4,Palm Beach County,26.677092,-80.563731,3.398141e-09,3.231395e-07,4.417846e-06,2.709678e-05,1.080439e-04,3.298795e-04,8.403096e-04,...,11.923197,41.420079,15.712627,21.398098,28.709236,37.970950,49.539330,63.796105,81.131285,101.899028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Laurens County,34.683402,-82.058502,0.000000e+00,0.000000e+00,0.000000e+00,1.182429e-12,1.898525e-10,4.058815e-09,4.091988e-08,...,9.265095,15.357635,16.873106,24.727228,35.554042,50.207683,69.685341,95.104257,127.644170,168.436932
130,Clarendon County,33.666562,-80.470185,0.000000e+00,0.000000e+00,0.000000e+00,9.341398e-12,1.499868e-09,3.194560e-08,3.042457e-07,...,20.699864,53.617502,23.525717,29.919382,37.294691,45.558695,54.560802,64.131877,74.149605,84.611741
131,Burke County,35.703545,-81.469485,1.815085e-09,1.748642e-07,2.576853e-06,1.760175e-05,7.831025e-05,2.641415e-04,7.356550e-04,...,43.631660,32.519321,5.611085,6.902817,8.283868,9.687003,11.032189,12.244395,13.280203,14.157456
132,Wayne County,35.438992,-78.063174,4.912997e-10,5.184675e-08,1.130786e-06,1.100857e-05,6.244102e-05,2.498056e-04,7.974522e-04,...,183.849460,124.396254,17.323228,21.003489,24.758215,28.304888,31.309365,33.443627,34.469798,34.328378


Drop columns 2011 to 2022

In [54]:
csi_Module_ia.drop(csi_Module_ia.loc[:, 2011:2023], inplace=True, axis=1)
csi_glass_ia.drop(csi_glass_ia.loc[:, 2011:2023], inplace=True, axis=1)
csi_silicon_ia.drop(csi_silicon_ia.loc[:, 2011:2023], inplace=True, axis=1)
csi_silver_ia.drop(csi_silver_ia.loc[:, 2011:2023], inplace=True, axis=1)
csi_copper_ia.drop(csi_copper_ia.loc[:, 2011:2023], inplace=True, axis=1)
csi_aluminium_frames_ia.drop(csi_aluminium_frames_ia.loc[:, 2011:2023], inplace=True, axis=1)
csi_encapsulant_ia.drop(csi_encapsulant_ia.loc[:, 2011:2023], inplace=True, axis=1)
csi_backsheet_ia.drop(csi_backsheet_ia.loc[:, 2011:2023], inplace=True, axis=1)

cdte_Module_ia.drop(cdte_Module_ia.loc[:, 2011:2023], inplace=True, axis=1)
cdte_cadmium_ia.drop(cdte_cadmium_ia.loc[:, 2011:2023], inplace=True, axis=1)
cdte_tellurium_ia.drop(cdte_tellurium_ia.loc[:, 2011:2023], inplace=True, axis=1)
cdte_glass_cdte_ia.drop(cdte_glass_cdte_ia.loc[:, 2011:2023], inplace=True, axis=1)
cdte_aluminium_frames_cdte_ia.drop(cdte_aluminium_frames_cdte_ia.loc[:, 2011:2023], inplace=True, axis=1)
cdte_copper_cdte_ia.drop(cdte_copper_cdte_ia.loc[:, 2011:2023], inplace=True, axis=1)
cdte_encapsulant_cdte_ia.drop(cdte_encapsulant_cdte_ia.loc[:, 2011:2023], inplace=True, axis=1)

Insert the 2023 column that summed the waste between 2011 to 2023.

In [55]:
csi_Module_ia.insert(3, 2023, csi_Module_ia_sumyears)
csi_glass_ia.insert(3, 2023, csi_glass_ia_sumyears)
csi_silicon_ia.insert(3, 2023, csi_silicon_ia_sumyears)
csi_silver_ia.insert(3, 2023, csi_silver_ia_sumyears)
csi_copper_ia.insert(3, 2023, csi_copper_ia_sumyears)
csi_aluminium_frames_ia.insert(3, 2023, csi_aluminium_frames_ia_sumyears)
csi_encapsulant_ia.insert(3, 2023, csi_encapsulant_ia_sumyears)
csi_backsheet_ia.insert(3, 2023, csi_backsheet_ia_sumyears)

cdte_Module_ia.insert(3, 2023, cdte_Module_ia_sumyears) 
cdte_cadmium_ia.insert(3, 2023, cdte_cadmium_ia_sumyears) 
cdte_tellurium_ia.insert(3, 2023, cdte_tellurium_ia_sumyears) 
cdte_glass_cdte_ia.insert(3, 2023, cdte_glass_cdte_ia_sumyears) 
cdte_aluminium_frames_cdte_ia.insert(3, 2023, cdte_aluminium_frames_cdte_ia_sumyears) 
cdte_copper_cdte_ia.insert(3, 2023, cdte_copper_cdte_ia_sumyears) 
cdte_encapsulant_cdte_ia.insert(3, 2023, cdte_encapsulant_cdte_ia_sumyears) 

#### 2.3.1. Change years for amounts

Change the column names as 'amount 1', 'amount 2', etc. I am not sure if it matters, but I am going to do it just in case!
From 2023 to 2050, we have a total of 28 amounts.

In [56]:
csi_Module_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
csi_glass_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
csi_silicon_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
csi_silver_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
csi_copper_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
csi_aluminium_frames_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
csi_encapsulant_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
csi_backsheet_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)

cdte_Module_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
cdte_cadmium_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
cdte_tellurium_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
cdte_glass_cdte_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
cdte_aluminium_frames_cdte_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
cdte_copper_cdte_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)
cdte_encapsulant_cdte_ia.set_axis(['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'], axis=1, inplace=True)


### 2.4. Create new PV datasets that add cSi and CdTe 

Here I add cSi and CdTe modules as one, and common materials as one.

In [57]:
pv_Modules_ia = pd.DataFrame(columns = ['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'])
pv_glass_ia = pd.DataFrame(columns = ['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'])
pv_copper_ia = pd.DataFrame(columns = ['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'])
pv_aluminium_frames_ia = pd.DataFrame(columns = ['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'])
pv_encapsulant_ia = pd.DataFrame(columns = ['name', 'latitude (deg)', 'longitude (deg)','amount 1','amount 2','amount 3','amount 4','amount 5','amount 6','amount 7','amount 8','amount 9','amount 10','amount 11','amount 12','amount 13','amount 14','amount 15','amount 16','amount 17','amount 18','amount 19','amount 20','amount 21','amount 22','amount 23','amount 24','amount 25','amount 26','amount 27','amount 28'])


Fill the data of name, latitude and longitude.

In [58]:
pv_Modules_ia['name'], pv_Modules_ia['latitude (deg)'],pv_Modules_ia['longitude (deg)'] = csi_Module_ia[['name']], csi_Module_ia[['latitude (deg)']],csi_Module_ia[['longitude (deg)']] 
pv_glass_ia['name'], pv_glass_ia['latitude (deg)'],pv_glass_ia['longitude (deg)'] = csi_Module_ia[['name']], csi_Module_ia[['latitude (deg)']],csi_Module_ia[['longitude (deg)']] 
pv_copper_ia['name'], pv_copper_ia['latitude (deg)'],pv_copper_ia['longitude (deg)'] = csi_Module_ia[['name']], csi_Module_ia[['latitude (deg)']],csi_Module_ia[['longitude (deg)']] 
pv_aluminium_frames_ia['name'], pv_aluminium_frames_ia['latitude (deg)'],pv_aluminium_frames_ia['longitude (deg)'] = csi_Module_ia[['name']], csi_Module_ia[['latitude (deg)']],csi_Module_ia[['longitude (deg)']] 
pv_encapsulant_ia['name'], pv_encapsulant_ia['latitude (deg)'],pv_encapsulant_ia['longitude (deg)'] = csi_Module_ia[['name']], csi_Module_ia[['latitude (deg)']],csi_Module_ia[['longitude (deg)']] 

Add amounts.

In [59]:
pv_Modules_ia.iloc[:,3:31] = cdte_Module_ia.iloc[:,3:31] + csi_Module_ia.iloc[:,3:31] 
pv_glass_ia.iloc[:,3:31] = csi_glass_ia.iloc[:,3:31] + cdte_glass_cdte_ia.iloc[:,3:31] 
pv_copper_ia.iloc[:,3:31] = csi_copper_ia.iloc[:,3:31] + cdte_copper_cdte_ia.iloc[:,3:31] 
pv_aluminium_frames_ia.iloc[:,3:31] = csi_aluminium_frames_ia.iloc[:,3:31] + cdte_aluminium_frames_cdte_ia.iloc[:,3:31] 
pv_encapsulant_ia.iloc[:,3:31] = csi_encapsulant_ia.iloc[:,3:31] + cdte_encapsulant_cdte_ia.iloc[:,3:31] 





In [60]:
pv_Modules_ia

,name,latitude (deg),longitude (deg),amount 1,amount 2,amount 3,amount 4,amount 5,amount 6,amount 7,...,amount 19,amount 20,amount 21,amount 22,amount 23,amount 24,amount 25,amount 26,amount 27,amount 28
0,Snohomish County,47.820991,-121.452252,3.925429,3.031427,4.906164,7.701479,11.764139,17.528199,25.519155,...,526.372741,592.005677,1168.746671,1153.040178,726.501890,1385.307512,1074.634355,1384.735295,2015.504538,2170.280550
1,state,35.120104,-117.159039,768.492353,558.537308,888.487822,1373.992869,2070.491795,3045.768148,4379.702342,...,56871.020353,48926.786422,136228.103101,116651.120738,27004.343537,65264.578495,26927.943241,30499.643617,52328.219326,31355.332116
2,Warren County,38.791108,-78.257714,0.019036,0.028956,0.064401,0.133228,0.258488,0.474220,0.828682,...,62.788883,119.629550,97.342019,108.430385,108.790955,331.299229,83.151303,92.469900,215.155566,77.818848
3,Sumter County,28.708695,-82.192477,39.580418,28.270457,45.065122,70.356854,107.810855,162.328102,240.281072,...,8635.272225,11441.145607,16525.472686,16970.118380,12981.454566,26144.087406,14433.631094,16223.615784,23790.143666,17298.148090
4,Palm Beach County,26.677092,-80.563731,17.986525,11.960300,18.424466,27.784920,41.127173,59.852677,85.735202,...,2773.754484,4737.173998,4892.003398,6121.300562,7367.991743,14628.671884,10365.141250,13135.792033,19337.343981,19519.428389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,Laurens County,34.683402,-82.058502,0.094862,0.154629,0.349845,0.732860,1.439312,2.682897,4.804529,...,1796.814921,2734.828228,4035.820639,5621.103798,7604.586779,11393.147067,14568.010122,19871.322770,27055.406603,35073.156622
130,Clarendon County,33.666562,-80.470185,0.486647,0.906378,2.239968,5.062903,10.584806,20.693775,38.200041,...,4656.488548,7104.412870,7891.526244,9317.188997,10266.695839,18479.631868,12697.061320,14691.607551,20085.656873,17635.866037
131,Burke County,35.703545,-81.469485,32.882405,25.027763,40.558980,63.886265,98.060643,146.948859,215.294200,...,4862.021751,4993.786881,10931.902195,9667.092490,3274.769245,8645.080924,2825.404890,3057.627063,5896.425211,2454.602499
132,Wayne County,35.438992,-78.063174,66.124012,57.043647,97.792729,162.277831,261.311118,409.193242,624.269613,...,19787.221098,19585.670679,45291.815501,39366.361036,11240.281916,31994.051713,8648.677267,9072.657957,19690.740412,5489.193752


### 2.5. Export the 'Ininital amounts' files

In [61]:
pv_Modules_ia.to_csv('RELOG_import_data/pv_Modules_ia.csv')
pv_glass_ia.to_csv('RELOG_import_data/pv_glass_ia.csv')
pv_copper_ia.to_csv('RELOG_import_data/pv_copper_ia.csv')
pv_aluminium_frames_ia.to_csv('RELOG_import_data/pv_aluminium_frames_ia.csv')
pv_encapsulant_ia.to_csv('RELOG_import_data/pv_encapsulant_ia.csv')

csi_Module_ia.to_csv('RELOG_import_data/csi_Module_ia.csv')
csi_glass_ia.to_csv('RELOG_import_data/csi_glass_ia.csv')
csi_silicon_ia.to_csv('RELOG_import_data/csi_silicon_ia.csv')
csi_silver_ia.to_csv('RELOG_import_data/csi_silver_ia.csv')
csi_copper_ia.to_csv('RELOG_import_data/csi_copper_ia.csv')
csi_aluminium_frames_ia.to_csv('RELOG_import_data/csi_aluminium_frames_ia.csv')
csi_encapsulant_ia.to_csv('RELOG_import_data/csi_encapsulant_ia.csv')
csi_backsheet_ia.to_csv('RELOG_import_data/csi_backsheet_ia.csv')

cdte_Module_ia.to_csv('RELOG_import_data/cdte_Module_ia.csv')
cdte_cadmium_ia.to_csv('RELOG_import_data/cdte_cadmium_ia.csv')
cdte_tellurium_ia.to_csv('RELOG_import_data/cdte_tellurium_ia.csv')
cdte_glass_cdte_ia.to_csv('RELOG_import_data/cdte_glass_cdte_ia.csv')
cdte_aluminium_frames_cdte_ia.to_csv('RELOG_import_data/cdte_aluminium_frames_cdte_ia.csv')
cdte_copper_cdte_ia.to_csv('RELOG_import_data/cdte_copper_cdte_ia.csv')
cdte_encapsulant_cdte_ia.to_csv('RELOG_import_data/cdte_encapsulant_cdte_ia.csv')


Remember that silver, silicon and backsheet are exclusively from cSi modules, likewise cadmium and tellurium are exclusive from CdTe. Further studies might not mix glass due to different compositions.

---
## 3. Make the recycling candidate file

Here we need to format the candidate location file as RELOG format. We are going to use Iloeje's file. This file was downloaded from [Iloeje's RELOG protocol](https://gcc02.safelinks.protection.outlook.com/?url=https%3A%2F%2Fzenodo.org%2Frecord%2F7093835&data=05%7C01%7CMacarena.MendezRibo%40nrel.gov%7Cff06eb8094844577193c08daf5ad67c6%7Ca0f29d7e28cd4f5484427885aee7c080%7C0%7C0%7C638092422384595963%7CUnknown%7CTWFpbGZsb3d8eyJWIjoiMC4wLjAwMDAiLCJQIjoiV2luMzIiLCJBTiI6Ik1haWwiLCJXVCI6Mn0%3D%7C3000%7C%7C%7C&sdata=WSieyEH9ngwW08cjlpBDggXqMZKiTjn9rrnFYbGGPac%3D&reserved=0) named 'CandidateLocations.csv', the file is the same for 'conservative' and 'optimistic'. 
Although this file is technically already with the right format, the area cost factor is set for Georgia, so we have to change this, and set it for California, because it is where we made the Recycling Plant's calculations.

### 3.1. States into region bins.

In [ ]:
from itertools import chain

In [ ]:
us_regions = {'New England' : set(['Connecticut', 'Maine', 'Massachusetts', 'New Hampshire', 'Rhode Island', 'Vermont']),
            'Middle Atlantic': set(['Delaware', 'the District of Columbia', 'Maryland', 'New Jersey', 'New York', 'Pennsylvania', 'Virginia', 'West Virginia']),
            'South Atlantic': set(['Georgia', 'North Carolina', 'South Carolina']),
            'Midwest': set(['Illinois', 'Indiana', 'Iowa', 'Kansas', 'Michigan', 'Minnesota', 'Missouri', 'Nebraska', 'North Dakota', 'Ohio', 'South Dakota', 'Wisconsin']),
            'Gulf': set(['Texas', 'Louisiana', 'Mississippi', 'Alabama', 'Florida', 'Arkansas', 'Kentucky', 'Tennessee']),
            'Southwest': set(['Nevada', 'Oklahoma', 'Utah']),
            'Mountain': set(['Arizona', 'Colorado', 'Idaho', 'Montana', 'New Mexico', 'Wyoming']),
            'Pacific Coast': set(['California', 'Oregon', 'Washington', 'Alaska', 'Hawaii'])}

### 3.2. Generate cost indexes

Relative labor rate and productivity indexes in the
chemical and allied products industries for the United States (1989). Source: PLANT DESIGN AND ECONOMICS FOR CHEMICAL ENGINEERS, Peter M. S.

In [ ]:
cost_index = {'Geographical area': ['New England', 'Middle Atlantic', 'South Atlantic', 'Midwest', 'Gulf', 'Southwest', 'Mountain', 'Pacific Coast'], 'Relative labor rate': [1.14, 1.06, 0.84, 1.03, 0.95, 0.88, 0.88, 1.22], 'Relative productivity factor': [0.95, 0.96, 0.91, 1.06, 1.22, 1.04, 0.97, 0.89]}
index_df = pd.DataFrame(data=cost_index)                                 

In [ ]:
index_df

Now I need to get the factor normalized with California (Pacific Coast), I get this calculation also from the book "Plant design and economics for..."

NOTE: I have to calculate the price for CdTe in California as well,  this might be a problem if we estimate the price based on FS's which are based in Ohio.

In [ ]:
pc_labor_rate = index_df.loc[index_df['Geographical area'] == 'Pacific Coast']['Relative labor rate'].values 
pc_prod_factor = index_df.loc[index_df['Geographical area'] == 'Pacific Coast']['Relative productivity factor'].values 

In [ ]:
pc_prod_factor

In [ ]:
index_df['Relative labor rate CA'], index_df['Relative productivity factor CA'] = index_df['Relative labor rate']/pc_labor_rate, index_df['Relative productivity factor']/pc_prod_factor

Now let's calculate the "Construction cost" or area factor:

In [ ]:
index_df['Area factor'] = index_df['Relative labor rate CA']/ index_df['Relative productivity factor CA']

In [ ]:
index_df

### 3.3. Generate the candidate location file

Load the Recycling plant's candidate locations. 

In [ ]:
cadidate_loc = pd.read_csv('CandidateLocations.csv')

In [ ]:
cadidate_loc

Let's create a new column with states so we can map the area cost factor with the right area.

In [ ]:
cadidate_loc['State'] = cadidate_loc['name'].str.rsplit(', ').str[-1] 

In [ ]:
cadidate_loc

---
#### 3.3.1. If no file is given, these cells shows how to set up the dataframe in RELOG format

Setup the template for the collection center file.

In [ ]:
candidate_locations = pd.DataFrame(columns=['name', 'latitude (deg)', 'longitude (deg)', 'area cost factor'])                

I am going to correlate states and regions as well and then delete it, this is to add the area cost factors.

In [ ]:
candidate_locations['name'], candidate_locations['state'], candidate_locations['latitude (deg)'], candidate_locations['longitude (deg)'] = GIS_usa['location'], GIS_usa['state'], GIS_usa['long'], GIS_usa['lat']

---
Make a dictionary of Geographical area (or regions) and area factors.

In [ ]:
index_dict = dict(zip(index_df['Geographical area'], index_df['Area factor']))

In [ ]:
index_dict

In [ ]:
us_regions.keys()

In [ ]:
for key in us_regions:
    cadidate_loc.loc[cadidate_loc['State'].isin(us_regions[key]), 'Region'] = key

In [ ]:
cadidate_loc

In [ ]:
cadidate_loc["area cost factor"] = cadidate_loc["Region"].apply(lambda x: index_dict.get(x))

Now we drop the state and Region columns:

In [ ]:
candidate_loc_clean = cadidate_loc.drop(['State', 'Region'], axis=1)

In [ ]:
candidate_loc_clean

In [ ]:
candidate_loc_clean['latitude (deg)'] = candidate_loc_clean['latitude (deg)'].round(decimals=4)
candidate_loc_clean['longitude (deg)'] = candidate_loc_clean['longitude (deg)'].round(decimals=4)
candidate_loc_clean['area cost factor'] = candidate_loc_clean['area cost factor'].round(decimals=2)

In [ ]:
candidate_loc_clean

In [ ]:
candidate_loc_clean.to_csv('RELOG_import_data/CandidateLocations.csv', index=False)

In [ ]:
# with pd.option_context('display.max_rows', None,
#                        'display.max_columns', None,
#                        'display.precision', 3,
#                        ):
#     print(candidate_locations)

---
## 4. Sankey Diagram

For the Sankey Diagram, I need:
1) Get the waste (cSi + CdTe).
    * Get the cSi waste and CdTe waste.
    * Get the amount of each material in cSi and CdTe.
2) Add all those materials into material bins.
3) Pass them by an intermediate bin with the recycling process an d  their recycling yield.
4) Add all those materials and check how much of each could contribute to revenue based on their value.

**Note:** To visualize the Sankey diagram in Jupyter, you may need to add the plotly extension, you can install it by running this line in the terminal: `jupyter labextension install plotlywidget`. [Source]().

### 4.0. Load the waste files if you haven't run the previous cells

In [2]:
# Uncomment if you need this option

csi_Module = pd.read_csv('csi_wasteEOL_Module.csv')
csi_aluminium_frames = pd.read_csv('csi_wasteEOL_aluminium_frames.csv')
csi_backsheet = pd.read_csv('csi_wasteEOL_backsheet.csv')
csi_copper = pd.read_csv('csi_wasteEOL_copper.csv')
csi_encapsulant = pd.read_csv('csi_wasteEOL_encapsulant.csv')
csi_glass = pd.read_csv('csi_wasteEOL_glass.csv')
csi_silicon = pd.read_csv('csi_wasteEOL_silicon.csv')
csi_silver = pd.read_csv('csi_wasteEOL_silver.csv')



cdte_Module = pd.read_csv('cdte_wasteEOL_Module.csv')
cdte_aluminium_frames_cdte = pd.read_csv('cdte_wasteEOL_aluminium_frames_cdte.csv')
cdte_cadmium = pd.read_csv('cdte_wasteEOL_cadmium.csv')
cdte_copper_cdte = pd.read_csv('cdte_wasteEOL_copper_cdte.csv')
cdte_encapsulant_cdte = pd.read_csv('cdte_wasteEOL_encapsulant_cdte.csv')
cdte_glass_cdte = pd.read_csv('cdte_wasteEOL_glass_cdte.csv')
cdte_tellurium = pd.read_csv('cdte_wasteEOL_tellurium.csv')

### 4.1. Get all waste

In [3]:
csi_waste = csi_Module['total waste'].sum()
cdte_waste = cdte_Module['total waste'].sum()
all_waste = csi_waste + cdte_waste
print(f'There are {all_waste:.2f} tonnes of PV waste (that\'s {all_waste/1000000:.2f} million metric tonnes).')
print(f'There are {csi_waste:.2f} tonnes of cSi, and {cdte_waste:.2f} tonnes of CdTe.')
perc_csi = csi_waste/all_waste
perc_cdte = cdte_waste/all_waste
print(f'Of all the waste, {perc_csi*100:.2f}% is cSi, and {perc_cdte*100:.2f}% is CdTe.')

There are 6659859.16 tonnes of PV waste (that's 6.66 million metric tonnes).
There are 5548525.34 tonnes of cSi, and 1111333.82 tonnes of CdTe.
Of all the waste, 83.31% is cSi, and 16.69% is CdTe.


In [4]:
# Option for one year, just change the year to the one you need

csi_waste_2050 = csi_Module['2050'].sum()
cdte_waste_2050 = cdte_Module['2050'].sum()
all_waste_2050 = csi_waste_2050 + cdte_waste_2050
print(f'There are {all_waste_2050:.2f} tonnes of PV waste (that\'s {all_waste_2050/1000000:.2f} million metric tonnes).')
print(f'There are {csi_waste_2050:.2f} tonnes of cSi, and {cdte_waste_2050:.2f} tonnes of CdTe.')
perc_csi_2050 = csi_waste_2050/all_waste_2050
perc_cdte_2050 = cdte_waste_2050/all_waste_2050
print(f'Of all the waste, {perc_csi_2050*100:.2f}% is cSi, and {perc_cdte_2050*100:.2f}% is CdTe.')

There are 491871.73 tonnes of PV waste (that's 0.49 million metric tonnes).
There are 388594.27 tonnes of cSi, and 103277.46 tonnes of CdTe.
Of all the waste, 79.00% is cSi, and 21.00% is CdTe.


### 4.2. Waste material bins

#### 4.2.1. cSi

In [5]:
csi_waste = {'Modules' : csi_Module['total waste'].sum(),
            'Glass' : csi_glass['total waste'].sum(),
            'Silicon' : csi_silicon['total waste'].sum(),
            'Silver': csi_silver['total waste'].sum(),
            'Copper' : csi_copper['total waste'].sum(),
            'Aluminium frames': csi_aluminium_frames['total waste'].sum(),
            'Encapsulant': csi_encapsulant['total waste'].sum(),
            'Backsheet': csi_backsheet['total waste'].sum(),}

In [6]:

csi_waste_2050 = {'Modules' : csi_Module['2050'].sum(),
            'Glass' : csi_glass['2050'].sum(),
            'Silicon' : csi_silicon['2050'].sum(),
            'Silver': csi_silver['2050'].sum(),
            'Copper' : csi_copper['2050'].sum(),
            'Aluminium frames': csi_aluminium_frames['2050'].sum(),
            'Encapsulant': csi_encapsulant['2050'].sum(),
            'Backsheet': csi_backsheet['2050'].sum(),}

#### 4.2.2. CdTe

In [7]:
cdte_waste = {'Modules' : cdte_Module['total waste'].sum(),
            'Glass' : cdte_glass_cdte['total waste'].sum(),
            'Cadmium': cdte_cadmium['total waste'].sum(),
            'Tellurium': cdte_tellurium['total waste'].sum(),
            'Copper' : cdte_copper_cdte['total waste'].sum(), # No data about the copper recovery, so I assume the same as FRELP
            'Aluminium frames': cdte_aluminium_frames_cdte['total waste'].sum(),
            'Encapsulant': cdte_encapsulant_cdte['total waste'].sum(),} # Here there is no info so I assume the same as the glass

In [8]:
# Option for one year, just change the year to the one you need

cdte_waste_2050 = {'Modules' : cdte_Module['2050'].sum(),
            'Glass' : cdte_glass_cdte['2050'].sum(),
            'Cadmium': cdte_cadmium['2050'].sum(),
            'Tellurium': cdte_tellurium['2050'].sum(),
            'Copper' : cdte_copper_cdte['2050'].sum(), # No data about the copper recovery, so I assume the same as FRELP
            'Aluminium frames': cdte_aluminium_frames_cdte['2050'].sum(),
            'Encapsulant': cdte_encapsulant_cdte['2050'].sum(),} # Here there is no info so I assume the same as the glass


### 4.3. Recycling bins <a id='#Section4.3'></a>

#### 4.3.1. cSi

In [9]:
#FRELP efficiencies unless indicated

csi_recycled = {'Modules' : csi_Module['total waste'].sum(),
            'Glass' : csi_glass['total waste'].sum()*0.98, 
            'Silicon' : csi_silicon['total waste'].sum()*0.95,
            'Silver': csi_silver['total waste'].sum()*0.95,
            'Copper' : csi_copper['total waste'].sum()*0.95,
            'Aluminium frames': csi_aluminium_frames['total waste'].sum(), # Assume 100% from the frames
            'Encapsulant': csi_encapsulant['total waste'].sum(), # Here the encapsulant is incinerated so, 100% goes out
            'Backsheet': csi_backsheet['total waste'].sum(),# Same as encapsulant
            'Landfill': csi_glass['total waste'].sum()*(1-0.98) + 
                csi_silicon['total waste'].sum()*(1-0.95) + 
                csi_silver['total waste'].sum()*(1-0.95) + 
                csi_copper['total waste'].sum()*(1-0.95),
            'Energy': csi_encapsulant['total waste'].sum()+csi_backsheet['total waste'].sum()} # Amount of waste that is burned and returned as energy 

In [10]:
# Option for one year, just change the year to the one you need

csi_recycled_2050 = {'Modules' : csi_Module['2050'].sum(),
            'Glass' : csi_glass['2050'].sum()*0.98, 
            'Silicon' : csi_silicon['2050'].sum()*0.95,
            'Silver': csi_silver['2050'].sum()*0.95,
            'Copper' : csi_copper['2050'].sum()*0.95,
            'Aluminium frames': csi_aluminium_frames['2050'].sum(), # Assume 100% from the frames
            'Encapsulant': csi_encapsulant['2050'].sum(), # Here the encapsulant is incinerated so, 100% goes out
            'Backsheet': csi_backsheet['2050'].sum(),# Same as encapsulant
            'Landfill': csi_glass['2050'].sum()*(1-0.98) + 
                csi_silicon['2050'].sum()*(1-0.95) + 
                csi_silver['2050'].sum()*(1-0.95) + 
                csi_copper['2050'].sum()*(1-0.95),
            'Energy': csi_encapsulant['2050'].sum()+csi_backsheet['2050'].sum()} 

#### 4.3.2. CdTe

In [11]:
# First Solar efficiencies unless indicated

cdte_recycled = {'Modules' : cdte_Module['total waste'].sum(),
            'Glass' : cdte_glass_cdte['total waste'].sum()*0.9,
            'Cadmium': cdte_cadmium['total waste'].sum()*0.95,
            'Tellurium': cdte_tellurium['total waste'].sum()*0.95,
            'Copper' : cdte_copper_cdte['total waste'].sum()*0.95, # No data about the copper recovery, so I assume the same as FRELP
            'Aluminium frames': cdte_aluminium_frames_cdte['total waste'].sum(),
            'Encapsulant': cdte_encapsulant_cdte['total waste'].sum()*0.9,# Here there is no info so I assume the same as the glass
            'Landfill':cdte_glass_cdte['total waste'].sum()*(1-0.9) +
                cdte_cadmium['total waste'].sum()*(1-0.95) +
                cdte_tellurium['total waste'].sum()*(1-0.95) +
                cdte_copper_cdte['total waste'].sum()*(1-0.95) +
                cdte_encapsulant_cdte['total waste'].sum()*(1-0.9),
            } 

In [12]:
# Option for one year, just change the year to the one you need

cdte_recycled_2050 = {'Modules' : cdte_Module['2050'].sum(),
            'Glass' : cdte_glass_cdte['2050'].sum()*0.9,
            'Cadmium': cdte_cadmium['2050'].sum()*0.95,
            'Tellurium': cdte_tellurium['2050'].sum()*0.95,
            'Copper' : cdte_copper_cdte['2050'].sum()*0.95, # No data about the copper recovery, so I assume the same as FRELP
            'Aluminium frames': cdte_aluminium_frames_cdte['2050'].sum(),
            'Encapsulant': cdte_encapsulant_cdte['2050'].sum()*0.9,# Here there is no info so I assume the same as the glass
            'Landfill':cdte_glass_cdte['2050'].sum()*(1-0.9) +
                cdte_cadmium['2050'].sum()*(1-0.95) +
                cdte_tellurium['2050'].sum()*(1-0.95) +
                cdte_copper_cdte['2050'].sum()*(1-0.95) +
                cdte_encapsulant_cdte['2050'].sum()*(1-0.9),
            } 

### 4.4. Generate figures

In [13]:
import plotly.graph_objects as go

In [14]:
if not os.path.exists("images"):
    os.mkdir("images")

In [15]:
my_colors = {'pvwaste':'rgba(255, 243, 217, 1)',
             'csi_blue': 'rgba(199, 219, 244,1)',
             'cdte_tiel': 'rgba(215, 250, 245, 1)',
             'product_green': 'rgba(217, 240, 217, 1)',
             'energy_yellow': 'rgba(252, 252, 202, 1)',
             'waste_red': 'rgba(247, 145, 116,1)',
             'worth_green': 'rgba(192, 232, 131,1)'}

In [16]:
material_list_csi = ['glass', 'silicon', 'silver', 'copper', 'aluminium_frames', 'encapsulant', 'backsheet', 'Module']
material_list_cdte = ['cadmium', 'telluride', 'glass_cdte', 'aluminium_frames_cdte', 'Module', 'copper_cdte', 'encapsulant_cdte']

#### 4.4.1. Sankey Option 1 - Labeled

In [17]:
fig = go.Figure(data=[go.Sankey(
    arrangement = "snap",
    node = dict(
      pad = 10,
      thickness = 20,
      line = dict(color = 'black', width = 0.5),
      label = ['PV Waste', 'cSi', 'CdTe', 
               'Glass', 'Silicon', 'Silver', 'Copper', 'Aluminum frames', 'Encapsulant', 'Backsheet', 
               'Glass', 'Cadmium', 'Tellurium', 'Copper', 'Aluminum frames', 'Encapsulant', 
               'cSi Recycling', 'CdTe Recycling', 
               'Glass scrap', 'Manufacturing grade silicon', 'Copper scrap', 'Silver scrap', 'Cadmium scrap', 'Tellurium scrap', 'Aluminum scrap', 'Landfill', 'Energy'],
      color = [my_colors['pvwaste'], my_colors['csi_blue'], my_colors['cdte_tiel'], 
               my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], 
               my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'], 
               my_colors['csi_blue'], my_colors['cdte_tiel'], 
               my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['waste_red'], my_colors['worth_green']]
),
    link = dict(
      source = [0, 0, 
                1, 1, 1, 1, 1, 1, 1, 
                2, 2, 2, 2, 2, 2, 
                3, 4, 5, 6, 7, 8, 9,
                10, 11, 12, 13, 14, 15,
                16,16,16,16,16,16,16,
                17,17,17,17,17,17, ], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2, 
                3, 4, 5, 6, 7, 8, 9, 
                10, 11, 12, 13, 14, 15,
                16, 16, 16, 16, 16, 16, 16,
                17, 17, 17, 17, 17, 17,
                18, 19, 20, 21, 24, 25, 26,
                18, 20, 22, 23, 24, 25],
      value = [csi_waste['Modules'], cdte_waste['Modules'], 
               csi_waste['Glass'], csi_waste['Silicon'], csi_waste['Silver'], csi_waste['Copper'], csi_waste['Aluminium frames'], csi_waste['Encapsulant'], csi_waste['Backsheet'], 
               cdte_waste['Glass'], cdte_waste['Cadmium'], cdte_waste['Tellurium'], cdte_waste['Copper'], cdte_waste['Aluminium frames'], cdte_waste['Encapsulant'],
               csi_waste['Glass'], csi_waste['Silicon'], csi_waste['Silver'], csi_waste['Copper'], csi_waste['Aluminium frames'], csi_waste['Encapsulant'], csi_waste['Backsheet'], 
               cdte_waste['Glass'], cdte_waste['Cadmium'], cdte_waste['Tellurium'], cdte_waste['Copper'], cdte_waste['Aluminium frames'], cdte_waste['Encapsulant'],
               csi_recycled['Glass'], csi_recycled['Silicon'], csi_recycled['Copper'], csi_recycled['Silver'], csi_recycled['Aluminium frames'], csi_recycled['Landfill'],csi_recycled['Energy'],
               cdte_recycled['Glass'], cdte_recycled['Copper'], cdte_recycled['Cadmium'], cdte_recycled['Tellurium'], cdte_recycled['Aluminium frames'], cdte_recycled['Landfill']],
      color = 'rgba(240, 240, 245, 0.65)'
  ))])

fig.update_layout(font_family="Times New Roman", font_size=10)
fig.write_image("images/sankey_labeled.svg")

#### 4.4.2. Sankey Option 2 - Labeled simplified

In [18]:
fig = go.Figure(data=[go.Sankey(
    arrangement = "snap",
    node = dict(
      pad = 10,
      thickness = 20,
      line = dict(color = 'black', width = 0.5),
      label = ['PV Waste', 'cSi Recycling', 'CdTe Recycling', 
               'Glass scrap', 'Manufacturing grade silicon', 'Copper scrap', 'Silver scrap', 'Cadmium scrap', 'Tellurium scrap', 'Aluminum scrap', 'Landfill', 'Energy'],
      color = [my_colors['pvwaste'], my_colors['csi_blue'], my_colors['cdte_tiel'], 
               my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['waste_red'], my_colors['worth_green']]
),
    link = dict(
      source = [0, 0, 
                1,1,1,1,1,1,1,
                2,2,2,2,2,2, ], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2,
                3, 4, 5, 6, 9, 10, 11,
                3, 5, 7, 8, 9, 10],
      value = [csi_waste['Modules'], cdte_waste['Modules'], 
               csi_recycled['Glass'], csi_recycled['Silicon'], csi_recycled['Copper'], csi_recycled['Silver'], csi_recycled['Aluminium frames'], csi_recycled['Landfill'],csi_recycled['Energy'],
               cdte_recycled['Glass'], cdte_recycled['Copper'], cdte_recycled['Cadmium'], cdte_recycled['Tellurium'], cdte_recycled['Aluminium frames'], cdte_recycled['Landfill']],
      color = 'rgba(240, 240, 245, 0.65)'
  ))])

fig.update_layout(font_family="Times New Roman", font_size=10)
fig.write_image("images/sankey_labeled_simplified.svg")

#### 4.4.3. Sankey Option 3 - Muted

In [26]:
fig = go.Figure(data=[go.Sankey(
    arrangement = "snap",
    node = dict(
      pad = 10,
      thickness = 20,
      line = dict(color = 'black', width = 0.5),
      label = ['', '', '', 
               '', '', '', '', '', '', '', 
               '', '', '', '', '', '', 
               '', '', 
               '', '', '', '', '', '', '', '', ''],
      color = [my_colors['pvwaste'], my_colors['csi_blue'], my_colors['cdte_tiel'], 
               my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], my_colors['csi_blue'], 
               my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'],my_colors['cdte_tiel'], 
               my_colors['csi_blue'], my_colors['cdte_tiel'], 
               my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['waste_red'], my_colors['worth_green']]
),
    link = dict(
      source = [0, 0, 
                1, 1, 1, 1, 1, 1, 1, 
                2, 2, 2, 2, 2, 2, 
                3, 4, 5, 6, 7, 8, 9,
                10, 11, 12, 13, 14, 15,
                16,16,16,16,16,16,16,
                17,17,17,17,17,17, ], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2, 
                3, 4, 5, 6, 7, 8, 9, 
                10, 11, 12, 13, 14, 15,
                16, 16, 16, 16, 16, 16, 16,
                17, 17, 17, 17, 17, 17,
                18, 19, 20, 21, 24, 25, 26,
                18, 20, 22, 23, 24, 25],
      value = [csi_waste['Modules'], cdte_waste['Modules'], 
               csi_waste['Glass'], csi_waste['Silicon'], csi_waste['Silver'], csi_waste['Copper'], csi_waste['Aluminium frames'], csi_waste['Encapsulant'], csi_waste['Backsheet'], 
               cdte_waste['Glass'], cdte_waste['Cadmium'], cdte_waste['Tellurium'], cdte_waste['Copper'], cdte_waste['Aluminium frames'], cdte_waste['Encapsulant'],
               csi_waste['Glass'], csi_waste['Silicon'], csi_waste['Silver'], csi_waste['Copper'], csi_waste['Aluminium frames'], csi_waste['Encapsulant'], csi_waste['Backsheet'], 
               cdte_waste['Glass'], cdte_waste['Cadmium'], cdte_waste['Tellurium'], cdte_waste['Copper'], cdte_waste['Aluminium frames'], cdte_waste['Encapsulant'],
               csi_recycled['Glass'], csi_recycled['Silicon'], csi_recycled['Copper'], csi_recycled['Silver'], csi_recycled['Aluminium frames'], csi_recycled['Landfill'],csi_recycled['Energy'],
               cdte_recycled['Glass'], cdte_recycled['Copper'], cdte_recycled['Cadmium'], cdte_recycled['Tellurium'], cdte_recycled['Aluminium frames'], cdte_recycled['Landfill']],
      color = 'rgba(240, 240, 245, 0.65)'
  ))])

fig.update_layout(font_family="Times New Roman", font_size=10)
fig.write_image("images/sankey_muted.svg")
#fig.write_image("images/sankey_mute.svg")

#### 4.4.4. Sankey Option 4 - Muted simplified

In [20]:
fig = go.Figure(data=[go.Sankey(
    arrangement = "snap",
    node = dict(
      pad = 10,
      thickness = 20,
      line = dict(color = 'black', width = 0.5),
      label = ['', '', '', 
               '', '', '', '', '', '', '', '', ''],
      color = [my_colors['pvwaste'], my_colors['csi_blue'], my_colors['cdte_tiel'], 
               my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['waste_red'], my_colors['worth_green']]
),
    link = dict(
      source = [0, 0, 
                1,1,1,1,1,1,1,
                2,2,2,2,2,2, ], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2,
                3, 4, 5, 6, 9, 10, 11,
                3, 5, 7, 8, 9, 10],
      value = [csi_waste['Modules'], cdte_waste['Modules'], 
               csi_recycled['Glass'], csi_recycled['Silicon'], csi_recycled['Copper'], csi_recycled['Silver'], csi_recycled['Aluminium frames'], csi_recycled['Landfill'],csi_recycled['Energy'],
               cdte_recycled['Glass'], cdte_recycled['Copper'], cdte_recycled['Cadmium'], cdte_recycled['Tellurium'], cdte_recycled['Aluminium frames'], cdte_recycled['Landfill']],
      color = 'rgba(240, 240, 245, 0.65)'
  ))])

fig.update_layout(font_family="Times New Roman", font_size=10)
fig.write_image("images/sankey_muted_simplified.svg")

In [27]:
# Figure for one year 

fig = go.Figure(data=[go.Sankey(
    arrangement = "snap",
    node = dict(
      pad = 10,
      thickness = 20,
      line = dict(color = 'black', width = 0.5),
      label = ['', '', '', 
               '', '', '', '', '', '', '', '', ''],
      color = [my_colors['pvwaste'], my_colors['csi_blue'], my_colors['cdte_tiel'], 
               my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['product_green'], my_colors['waste_red'], my_colors['worth_green']],
      hovertemplate= 'Node value is %{value}'
    ),
    link = dict(
      source = [0, 0, 
                1,1,1,1,1,1,1,
                2,2,2,2,2,2, ], # indices correspond to labels, eg A1, A2, A1, B1, ...
      target = [1, 2,
                3, 4, 5, 6, 9, 10, 11,
                3, 5, 7, 8, 9, 10],
      value = [csi_waste_2050['Modules'], cdte_waste_2050['Modules'], 
               csi_recycled_2050['Glass'], csi_recycled_2050['Silicon'], csi_recycled_2050['Copper'], csi_recycled_2050['Silver'], csi_recycled_2050['Aluminium frames'], csi_recycled_2050['Landfill'],csi_recycled_2050['Energy'],
               cdte_recycled_2050['Glass'], cdte_recycled_2050['Copper'], cdte_recycled_2050['Cadmium'], cdte_recycled_2050['Tellurium'], cdte_recycled_2050['Aluminium frames'], cdte_recycled_2050['Landfill']],
      color = 'rgba(240, 240, 245, 0.65)',
      hovertemplate= 'Link value is %{value}' 
  ))])

fig.update_layout(font_family="Times New Roman", font_size=10)
fig.write_image("images/sankey_muted_simplified_2050.png")
fig.show()

In [27]:
# Here I am checking the quantities of each to see how much of each material we have

In [28]:
# Every year cSi

('Glass', csi_recycled['Glass']), ('Silicon', csi_recycled['Silicon']), ('Copper', csi_recycled['Copper']), ('Silver', csi_recycled['Silver']), ('Aluminium frames', csi_recycled['Aluminium frames']), ('Landfill', csi_recycled['Landfill']),('Energy', csi_recycled['Energy'])


(('Glass', 3975173.2868785267),
 ('Silicon', 183330.20404407431),
 ('Copper', 3111.2586608322918),
 ('Silver', 2094.2163446579657),
 ('Aluminium frames', 685664.5362342701),
 ('Landfill', 91048.9159227935),
 ('Energy', 608102.9191697116))

In [33]:
# Every year CdTe
('Glass', cdte_recycled['Glass']), ('Copper', cdte_recycled['Copper']), ('Cadmium', cdte_recycled['Cadmium']), ('Tellurium', cdte_recycled['Tellurium']), ('Aluminium frames', cdte_recycled['Aluminium frames']), ('Landfill', cdte_recycled['Landfill']), ('Encapsulant', cdte_recycled['Encapsulant'])


(('Glass', 952329.5911787425),
 ('Copper', 3660.481848104059),
 ('Cadmium', 390.2071187928947),
 ('Tellurium', 459.77392815562615),
 ('Aluminium frames', 21998.061209019204),
 ('Landfill', 108696.18289739163),
 ('Encapsulant', 23799.519842231224))

In [30]:
# 2050

In [31]:
('Glass', csi_recycled_2050['Glass']), ('Silicon', csi_recycled_2050['Silicon']), ('Copper', csi_recycled_2050['Copper']), ('Silver', csi_recycled_2050['Silver']), ('Aluminium frames', csi_recycled_2050['Aluminium frames']), ('Landfill', csi_recycled_2050['Landfill']),('Energy', csi_recycled_2050['Energy'])


(('Glass', 292989.58565324114),
 ('Silicon', 11337.950055421034),
 ('Copper', 247.28803278563933),
 ('Silver', 94.07854397627003),
 ('Aluminium frames', 39847.661342862804),
 ('Landfill', 6594.080174423792),
 ('Energy', 37483.62377093767))

In [34]:
('Glass', cdte_recycled_2050['Glass']), ('Copper', cdte_recycled_2050['Copper']), ('Cadmium', cdte_recycled_2050['Cadmium']), ('Tellurium', cdte_recycled_2050['Tellurium']), ('Aluminium frames', cdte_recycled_2050['Aluminium frames']), ('Landfill', cdte_recycled_2050['Landfill']), ('Encapsulant', cdte_recycled_2050['Encapsulant'])



(('Glass', 85251.28813960454),
 ('Copper', 342.15082595687767),
 ('Cadmium', 35.511926793397805),
 ('Tellurium', 45.06502828182493),
 ('Aluminium frames', 5637.077841494511),
 ('Landfill', 9741.789451525477),
 ('Encapsulant', 2224.5774488990237))

---
## 5. Cloropeth map option 1 (still under construction)

In [ ]:
pv_waste_map = csi_Module[['FIPS']].copy()
pv_waste_map['total waste'] = csi_Module['total waste'] + cdte_Module['total waste']
pv_waste_map.to_csv('pv_waste_map.csv')

csi_waste_map = csi_Module[['FIPS', 'total waste']]
csi_waste_map.to_csv('csi_waste_map.csv')

cdte_waste_map = cdte_Module[['FIPS', 'total waste']]
cdte_waste_map.to_csv('cdte_waste_map.csv')

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("pv_waste_map.csv",
                   dtype={"FIPS": str})

import plotly.express as px

fig = px.choropleth_mapbox(df, geojson=counties, locations='FIPS', color='total waste',
                           color_continuous_scale="Viridis",
                           range_color=(8, 1200000), # min an max values of waste
                           mapbox_style="carto-positron",
                           zoom=2.5, center = {"lat": 37.0902, "lon": -95.7129},
                           opacity=0.5,
                           labels={'total waste':'Accumulated waste'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image("images/map_allPV_op1.svg")

Cloropeth map option 2

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("pv_waste_map.csv",
                   dtype={"FIPS": str})

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='FIPS', color='total waste',
                           color_continuous_scale="Viridis",
                           range_color=(1000, 50000),
                           scope="usa",
                           labels={'total waste':'Total PV waste by 2050'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image("images/map_allPV_op2.svg")

In [ ]:
pv_waste_map.min()

In [ ]:
pv_waste_map.max()

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/fips-unemp-16.csv",
                   dtype={"fips": str})

import plotly.express as px

fig = px.choropleth(df, geojson=counties, locations='fips', color='unemp',
                           color_continuous_scale="Viridis",
                           range_color=(0, 12),
                           scope="usa",
                           labels={'unemp':'unemployment rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.write_image("images/map_allPV_op2.svg")
